# 3D Scalar Advection Solver on a Sphere Using Radial Basis Functions

Samm Elliott

### The PDE
The transport equation for a passive tracer with mixing ratio $q$, without sources or sinks, can be written in the conservative flux form given by

$$\frac{\partial(\rho q)}{\partial t} + \nabla\cdot(\rho q \bar{u}) = 0.$$
$$\frac{\partial{q}}{\partial t} = - \bar{u}\cdot\nabla{q} .$$

where $\rho$ is the air density, $\bar{u}$ is the wind field and $\nabla$ is the standard 3D gradient operator. The mass continuity equation is given by

$$\frac{\partial\rho}{\partial t} + \nabla\cdot(\rho \bar{u}) = 0.$$

### Import Libraries

In [1]:
import numpy as np
from numpy import linalg as nla
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d.axes3d import Axes3D
import scipy.sparse as ssparse
from scipy.sparse import linalg as sla
import time
import netCDF4 as nc4
import os

# Physical Constants

In [2]:
a = 6.37122*(10**6)  # radius of the Earth (m)
g = 9.80616          # gravitational constant (m s^(-2))
p0 = 100000.0        # reference surface pressure (Pa)
cp = 1004.5          # specific heat capacity of dry air (J kg^(-1) K^(-1))
Rd = 287.0           # gas constant for dry air (J kg^(-1) K^(-1))
kappa = Rd/cp        # ratio of Rd to cp
T0 = 300.0           # isothermal atmosphere temperature (K)
pi = np.pi           # pi

### Simulation Parameterizations

In [27]:
htop = 12000.0       # height position of model top (m)
Nh = 256            # number of horizontal nodes
# Nh = 12100            # number of horizontal nodes
# Nv = 12              # number of vertical nodes
Nv = 30              # number of vertical nodes
n = 55               # RBF stencil size
dt = 1800

### Initializing the Nodeset

We first read in the specified maximal determinant (MD) nodes and add vertical levels for our nodeset. These MD nodes were aquired from http://web.maths.unsw.edu.au/~rsw/Sphere/. Check ./md directory for available nodesets.

In [28]:
# read in MD nodes - note these are for a unit sphere
X_hat = np.loadtxt("../../nodesets/md."+str(Nh).zfill(5),usecols = (0,1,2)).T

If $\hat{x}$ is a MD nodepoint on a unit sphere, then the corresponding point $\bar{x}$ in our domain for vertical level $n$ is given by

$$\bar{x} = (a+n\frac{z_{top}}{N_v})\hat{x}$$

In [29]:
# create nodeset
h = htop/(Nv-1)
rlvls = a + np.linspace(-3*h, htop + 3*h, num = Nv + 6)
X = np.tensordot(X_hat, rlvls, 0)

The following plots the MD nodes and the nodeset for our domain.

In [30]:
# fig = plt.figure(figsize=(8,8))
# ax = fig.add_subplot(1, 1, 1, projection='3d')
# ax.scatter(md_nodes[0],md_nodes[1],md_nodes[2])

In [31]:
# fig = plt.figure(figsize=(20,20))
# ax = fig.add_subplot(1, 1, 1, projection='3d')
# ax.scatter(xx[0],xx[1],xx[2])

### Creating n-point Stencils

We can just use the MD nodeset since the stencils will be valid for any radially scaled version of the nodeset. 

In [32]:
%run RBFFD_Generation.ipynb
%time DM1h3,DM1v3,Lh,idx = get_RBFFD_DMs(X_hat,n)
# gamma = 145*Nh**-4
depth = 3

get_RBFFD_DMs progress: %1
get_RBFFD_DMs progress: %20
get_RBFFD_DMs progress: %40
get_RBFFD_DMs progress: %60
get_RBFFD_DMs progress: %80
get_RBFFD_DMs progress: %100
CPU times: user 1.27 s, sys: 137 ms, total: 1.41 s
Wall time: 356 ms


In [35]:
# fig = plt.figure(figsize=(12,12))
# ax = fig.add_subplot(1, 1, 1, projection='3d')
# ax.scatter(X_hat[0],X_hat[1],X_hat[2],marker='o',c='blue',s=10)
# ax.scatter(X_hat[0,idx[0]],X_hat[1,idx[0]],X_hat[2,idx[0]],marker='o',c='green',s=50)
# ax.view_init(60,30)
print(idx[0:10])

[[  0  61  78  43  19  56 138  13 221 241  80 141  67 224 193  84 195  17
    1  68  91  55  31  60 181 107 152 215 153 183 142  98 196 239  23  30
   72 166 115  96 160   8  66 231  92 121  70  37 245 214  99  16  24 226
  250]
 [  1  23 141  68 152 115 138 193  92  17  70  43  26 237 196 169 239  78
    0 221 107  67 247 215 226 165 161 250  61 231 144 131 124 214  19 241
   91 224  31 113 111 156  56 166 198  72 157 202 192 135  80 137 120  84
  117]
 [  2  69 125 170   9 114 212  82 177 122 234 222 180  33 213  87 154 105
  132  54 101  28 108 119 171 109  49  36 253 112 163 118 227 162  63  73
   64 173 133 174  25 130 104  94 110  42  41 255 219  47  79 220 248 126
  208]
 [  3 199 240 252  18 201 233  90  53  88 197 134 209 244  50  29  44 184
  176  52 172  65 207 146   5 232 243 236 106  48  12  34 167  15 136  89
  158  62  58 148  35  39 147 188 228  76  20 117 100 194 206  57 186 157
   83]
 [  4 217 127  59 102  16  37 143   6   8 168  98 205 179 151 126  99 203
   60  32 

In [43]:
print(X_hat[2])

[ 1.          0.83108862 -0.44683909 -0.60347735  0.43087357 -0.49836962
  0.07453776 -0.21214761  0.68199281 -0.61230879 -0.15564701  0.41152862
 -0.81149373  0.92035224  0.32475254 -0.00307748  0.57643643  0.88086922
 -0.77850819  0.97365906 -0.85988069 -0.99490422  0.32061364  0.71012422
  0.57498711 -0.56940204  0.52483322  0.10770832 -0.0353995  -0.79314778
  0.70230263  0.81845825  0.41076227 -0.05561205 -0.15359755 -0.98621221
 -0.68757541  0.65215082  0.05502254  0.14810678 -0.45717055  0.35594933
 -0.74519692  0.97369287 -0.26144259 -0.56415916 -0.72960943 -0.94383104
 -0.91529409 -0.19091592 -0.43067484 -0.1520326  -0.68069889 -0.56283491
 -0.06480253  0.82378993  0.96871831 -0.95681311  0.13296667  0.30257088
  0.81734705  0.97754352 -0.61541081  0.17520687  0.03066516 -0.95080796
  0.67665526  0.90733312  0.83060722 -0.60973337  0.65733717  0.08069509
  0.69688487 -0.90207983 -0.01132352  0.25565628 -0.40092285  0.53350864
  0.97746175 -0.48984534  0.91072416 -0.49913054 -0

In [38]:
print((X_hat[0,0]-X_hat[0,idx[0,0]])**2 + (X_hat[1,0]-X_hat[1,idx[0]])**2 + (X_hat[2,0]-X_hat[2,idx[0]])**2)

[ 0.          0.04243415  0.03536544  0.01933596  0.03042349  0.00487733
  0.00866395  0.01867694  0.14682314  0.15628555  0.0977756   0.02392908
  0.16922279  0.18227657  0.09962043  0.13379011  0.03547829  0.12637138
  0.02853105  0.07814197  0.35125362  0.1001723   0.36287332  0.03503146
  0.30023031  0.30692226  0.10028541  0.32697218  0.353788    0.17665878
  0.29998555  0.09560309  0.25513055  0.30270731  0.0968748   0.58616218
  0.58606112  0.58546048  0.11459182  0.62032612  0.40658419  0.13811311
  0.26393528  0.51208498  0.23271959  0.58787467  0.25308727  0.12159852
  0.60109076  0.59171788  0.32828351  0.23390582  0.53574611  0.4990731
  0.5461506 ]


## Test Case 2: Hadley-like Circulation

In [10]:
# %run TC1_3D_deformational_flow.ipynb
# TC = 1
# plttype = 0

In [11]:
%run TC2_Hadley_circulation.ipynb
TC = 2
plttype = 1

In [12]:
r,lmbda,phi = c2s(X[0],X[1],X[2])

## Timestepping

In [13]:
def set_ghosts(var,method):
    
    for i in range(1,depth+1):
        
        if method == 0:
            var[:,depth - i] = 0.0
            var[:,(Nv + depth - 1) + i] = 0.0
            
        if method == 1:
            var[:,depth - i] = var[:,depth + i]
            var[:,(Nv + depth - 1) + i] = var[:,(Nv + depth - 1) - i]
        
        if method == 2:
            var[:,depth - i] = 2*var[:,depth] - var[:,depth + i]
            var[:,(Nv + depth - 1) + i] = 2*var[:,Nv + depth - 1] - var[:,(Nv + depth - 1) - i]
            
    return var

In [14]:
def eval_hyperviscosity(f,L,idx):
    
    # initialize gradient component arrays
    Lf = np.empty((Nh,Nv))
    
    ### evaluate horizontal components
    for i in range(Nh):
        
        L_i = L[i]
        
        for j in range(depth,Nv+depth):
            
            f_nbrs = f[idx[i],j]

            Lf[i,j-3] = np.vdot(f_nbrs,L_i)

    return Lf

def eval_gradient(f,DM1h3,DM1v3,idx):
    
    # horizontal/vertical differentiation scaling factors
    c_h = 1/rlvls
    c_v = 1/h
    
    # initialize gradient component arrays
    df_dx = np.empty((Nh,Nv))
    df_dy = np.empty((Nh,Nv))
    df_dz = np.empty((Nh,Nv))
    
    ### evaluate horizontal components
    for i in range(Nh):
        
        Dx_i = DM1h3[0,i]
        Dy_i = DM1h3[1,i]
        Dz_i = DM1h3[2,i]
        
        for j in range(depth,Nv+depth):
            
            f_nbrs = f[idx[i],j]
            
            df_dx[i,j-3] = np.vdot(f_nbrs,Dx_i)*c_h[j]
            df_dy[i,j-3] = np.vdot(f_nbrs,Dy_i)*c_h[j]
            df_dz[i,j-3] = np.vdot(f_nbrs,Dz_i)*c_h[j]
    
    ### evaluate vertical components
    for i in range(Nh):
        
        Dx_i = DM1v3[0,i]
        Dy_i = DM1v3[1,i]
        Dz_i = DM1v3[2,i]
        
        for j in range(depth,Nv+depth):
            
            f_nbrs = f[i,j-3:j+4]
            
            df_dx[i,j-3] += np.vdot(f_nbrs,Dx_i)*c_v
            df_dy[i,j-3] += np.vdot(f_nbrs,Dy_i)*c_v
            df_dz[i,j-3] += np.vdot(f_nbrs,Dz_i)*c_v
    
    return df_dx,df_dy,df_dz

def eval_RHS(q,U,DM1h3,DM1v3,Lh,idx):

    q = set_ghosts(q,1)
    
    dq_dx,dq_dy,dq_dz = eval_gradient(q,DM1h3,DM1v3,idx)
    
    Lq = eval_hyperviscosity(q,Lh,idx)
    
    gamma = -(1.5e4/a)*Nh**-4
    
    RHS = - ((U[0]*dq_dx) + (U[1]*dq_dy) + (U[2]*dq_dz)) + gamma*Lq
    
    return RHS


In [15]:
##### 1st order DM tests

# ### Test 1
# z = (r-a)
# ff = (z/htop)**2
# ff_x1 = ((2*X[0]*z)/(r*(htop**2)))[:,depth:Nv+depth]
# ff_y1 = ((2*X[1]*z)/(r*(htop**2)))[:,depth:Nv+depth]
# ff_z1 = ((2*X[2]*z)/(r*(htop**2)))[:,depth:Nv+depth]

# ### Test 2
# ff = np.sin(X[0]/a + pi/4) * np.sin(X[1]/a + pi/4) * np.sin(X[2]/a + pi/4) * a
# ff_x1 = ((np.cos(X[0]/a + pi/4) * np.sin(X[1]/a + pi/4) * np.sin(X[2]/a + pi/4)))[:,depth:Nv+depth]
# ff_y1 = ((np.sin(X[0]/a + pi/4) * np.cos(X[1]/a + pi/4) * np.sin(X[2]/a + pi/4)))[:,depth:Nv+depth]
# ff_z1 = ((np.sin(X[0]/a + pi/4) * np.sin(X[1]/a + pi/4) * np.cos(X[2]/a + pi/4)))[:,depth:Nv+depth]


# ### Evaluate Diffs
# print("Testing DMs:")
# ff_x2 = rbffd_diff_h(ff,DM1h3[0],idx,1) + rbffd_diff_v(ff,DM1v3[0],1)
# ff_y2 = rbffd_diff_h(ff,DM1h3[1],idx,1) + rbffd_diff_v(ff,DM1v3[1],1)
# ff_z2 = rbffd_diff_h(ff,DM1h3[2],idx,1) + rbffd_diff_v(ff,DM1v3[2],1)
# print("\tf:","\n\t\tmin = {0:e}".format(np.min(ff)),"\n\t\tmax = {0:e}".format(np.max(ff)))
# print("\tf_x:","\n\t\tmax(f_x) = {0:e}".format(np.max(np.abs(ff_x1))),"\n\t\tmax_err(f_x) = {0:e}".format(np.max(np.abs(ff_x1-ff_x2))),"\n\t\tmedian_err(f_x) = {0:e}".format(np.median(np.abs(ff_x1-ff_x2))),"\n\t\tave_err(f_x) = {0:e}".format(np.mean(np.abs(ff_x1-ff_x2))))
# print("\tf_y:","\n\t\tmax(f_y) = {0:e}".format(np.max(np.abs(ff_y1))),"\n\t\tmax_err(f_y) = {0:e}".format(np.max(np.abs(ff_y1-ff_y2))),"\n\t\tmedian_err(f_y) = {0:e}".format(np.median(np.abs(ff_y1-ff_y2))),"\n\t\tave_err(f_y) = {0:e}".format(np.mean(np.abs(ff_y1-ff_y2))))
# print("\tf_z:","\n\t\tmax(f_z) = {0:e}".format(np.max(np.abs(ff_z1))),"\n\t\tmax_err(f_z) = {0:e}".format(np.max(np.abs(ff_z1-ff_z2))),"\n\t\tmedian_err(f_z) = {0:e}".format(np.median(np.abs(ff_z1-ff_z2))),"\n\t\tave_err(f_z) = {0:e}".format(np.mean(np.abs(ff_z1-ff_z2))))


In [16]:
def plot_var(var):
    
    plt.ion()
    
    if plttype == 0:
        fig = plt.figure(figsize=(10,6))
        ax = fig.add_subplot(1,1,1)
        pltlvl = int(np.floor(5000/(2*h))+3)
        cax = ax.scatter(lmbda[:,0],phi[:,0],c = var[:,pltlvl])
        cbar = fig.colorbar(cax)
        
    if plttype == 1:
        fig = plt.figure(figsize=(16,6))
        ax = fig.add_subplot(1,1,1)
        cond = np.abs(lmbda-pi) < .1
        cax = ax.scatter(np.extract(cond,phi),np.extract(cond,r)-a,c = np.extract(cond,var),s=1.5e3/Nv)
        cbar = fig.colorbar(cax)
        
    plt.show()

In [17]:
nsteps = int(tau/dt)
plot_int = 1000

q_init = set_ghosts(get_q_init(r,lmbda,phi),0)
q = np.copy(q_init)
q_temp = np.copy(q_init)

hist_int = 1
q_hist = q_init.reshape((1,Nh,Nv+6))
hist_step = 0

for tstep in range(nsteps):
    
    start = time.time()
        
    # RK4 step 1
    t = tstep*dt
    U = u_t(r,lmbda,phi,t)[:,:,depth:Nv+depth]
    RHS = eval_RHS(q,U,DM1h3,DM1v3,Lh,idx)
    d = RHS
    
    # RK4 step 2
    t += dt/2
    U = u_t(r,lmbda,phi,t)[:,:,depth:Nv+depth]
    q_temp[:,depth:Nv+depth] = q[:,depth:Nv+depth] + (dt/2)*RHS
    RHS = eval_RHS(q_temp,U,DM1h3,DM1v3,Lh,idx)
    d += 2*RHS
    
    # RK4 step 3
    q_temp[:,depth:Nv+depth] = q[:,depth:Nv+depth] + (dt/2)*RHS
    RHS = eval_RHS(q_temp,U,DM1h3,DM1v3,Lh,idx)
    d += 2*RHS
    
    # RK4 step 4
    t += dt/2
    U = u_t(r,lmbda,phi,t)[:,:,depth:Nv+depth]
    q_temp[:,depth:Nv+depth] = q[:,depth:Nv+depth] + dt*RHS
    RHS = eval_RHS(q_temp,U,DM1h3,DM1v3,Lh,idx)
    d += RHS
    
    # Update tracers
    q[:,depth:Nv+depth] += (dt/6)*d
    
    t_total = time.time() - start
    
    
    print("Finished time step",tstep+1,"of",nsteps,"total -> {0:.1f} seconds/timestep".format(t_total))
    
    if (tstep+1)%plot_int == 0:
        plot_var(q)
    
    if (tstep+1)%hist_int == 0:
        hist_step += 1
        q_hist = np.append(q_hist,q.reshape((1,Nh,Nv+6)),axis=0)
        

Finished time step 1 of 48 total -> 30.6 seconds/timestep
Finished time step 2 of 48 total -> 30.2 seconds/timestep
Finished time step 3 of 48 total -> 30.7 seconds/timestep
Finished time step 4 of 48 total -> 31.1 seconds/timestep
Finished time step 5 of 48 total -> 30.1 seconds/timestep
Finished time step 6 of 48 total -> 31.2 seconds/timestep
Finished time step 7 of 48 total -> 30.2 seconds/timestep
Finished time step 8 of 48 total -> 33.3 seconds/timestep
Finished time step 9 of 48 total -> 33.5 seconds/timestep
Finished time step 10 of 48 total -> 32.5 seconds/timestep
Finished time step 11 of 48 total -> 33.4 seconds/timestep
Finished time step 12 of 48 total -> 31.9 seconds/timestep
Finished time step 13 of 48 total -> 29.6 seconds/timestep
Finished time step 14 of 48 total -> 29.1 seconds/timestep
Finished time step 15 of 48 total -> 30.2 seconds/timestep
Finished time step 16 of 48 total -> 30.0 seconds/timestep
Finished time step 17 of 48 total -> 31.2 seconds/timestep
Finish

In [23]:
times = np.linspace(0,tau,num = hist_step+1)

In [24]:
outputFile = "results/TC"+str(TC)+"_"+str(Nh)+"h_"+str(Nv)+"v.nc"
os.remove(outputFile)
ncf_root = nc4.Dataset(outputFile,"w","NETCDF4")
ncd_time = ncf_root.createDimension("tstep", hist_step+1)
ncd_hid = ncf_root.createDimension("hid", Nh)
ncd_vid = ncf_root.createDimension("vid", Nv+6)
ncv_r = ncf_root.createVariable("r","f8",("hid","vid"))
ncv_lmbda = ncf_root.createVariable("lmbda","f8",("hid","vid"))
ncv_phi = ncf_root.createVariable("phi","f8",("hid","vid"))
ncv_time = ncf_root.createVariable("times","f8",("tstep"))
ncv_q_hist = ncf_root.createVariable("q_hist","f8",("tstep","hid","vid"))
ncf_root.variables["r"][:] = r[:]
ncf_root.variables["lmbda"][:] = lmbda
ncf_root.variables["phi"][:] = phi
ncf_root.variables["times"][:] = times
ncf_root.variables["q_hist"][:] = q_hist
ncf_root.close()

In [25]:
### Extrema
q_min = np.min(q)
q_max = np.max(q)

### Solution Error Norms
l1 = np.sum(np.abs(q_init - q)) / np.sum(np.abs(q_init))
l2 = np.sqrt(np.sum((q_init - q)**2) / np.sum(q_init**2))
linf = np.max(np.abs(q_init - q)) / np.max(np.abs(q_init))

### Mass Conservation
rho_0 = rho(r)
V = (4*pi*r**2)*h/Nh
Mt_tot = np.sum(rho_0*V*q)
M0_tot = np.sum(rho_0*V*q_init)

### Print Error Results
print("Solution Extrema:\n\tMin:  \t{0:.2e}".format(q_min),"\n\tMax:  \t{0:.2e}".format(q_max))
print("Error Results:\n\tL_1:  \t{0:.2e}".format(l1),"\n\tL_2:  \t{0:.2e}".format(l2),"\n\tL_inf:\t{0:.2e}".format(linf))
print("\nTracer Mass Difference:\t%{0:.3f}".format(np.abs(100*(Mt_tot-M0_tot)/M0_tot)))


Solution Extrema:
	Min:  	-5.64e-02 
	Max:  	9.75e-01
Error Results:
	L_1:  	3.69e-02 
	L_2:  	3.30e-02 
	L_inf:	9.27e-02

Tracer Mass Difference:	%0.005


In [26]:
# Config: Nh = 4096, Nv = 30, n = 55, dt = 1800, 
# Extrema: Min = -.21, Max = 1.08 --- at tau/2: q_min = -0.72, q_max = 1.88
# Error: L_1 = .083, L_2 = .077, L_inf = .26
# Tracer Mass Difference: %.87
# Timing: 10.5 seconds/timestep

# Config: Nh = 12100, Nv = 30, n = 55, dt = 1800, 
# Extrema: Min = -.056, Max = .96 
# Error: L_1 = .037, L_2 = .033, L_inf = .093
# Tracer Mass Difference: %0.005
# Timing: 27 seconds/timestep
